In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import pandas as pd

books = pd.read_csv('df_cleaned.csv')

In [5]:
books["tagged_description"]

0       9780002005883A NOVEL THAT READERS and critics ...
1       9780002261982A new 'Christie for Christmas' --...
2       9780006178736A memorable, mesmerizing heroine ...
3       9780006280897Lewis' work on the nature of love...
4       9780006280934"In The Problem of Pain, C.S. Lew...
                              ...                        
5192    9788172235222On A Train Journey Home To North ...
5193    9788173031014This book tells the tale of a man...
5194    9788179921623Wisdom to Create a Life of Passio...
5195    9788185300535This collection of the timeless t...
5196    9789027712059Since the three volume edition of...
Name: tagged_description, Length: 5197, dtype: object

In [6]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False,
                                   header = False)

In [7]:
raw_doc = TextLoader("tagged_description.txt",encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0,chunk_overlap=0,separator="\n")
doc = text_splitter.split_documents(raw_doc)

Created a chunk of size 1167, which is longer than the specified 0
Created a chunk of size 1213, which is longer than the specified 0
Created a chunk of size 372, which is longer than the specified 0
Created a chunk of size 308, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 481, which is longer than the specified 0
Created a chunk of size 959, which is longer than the specified 0
Created a chunk of size 187, which is longer than the specified 0
Created a chunk of size 842, which is longer than the specified 0
Created a chunk of size 295, which is longer than the specified 0
Created a chunk of size 196, which is longer than the specified 0
Created a chunk of size 880, which is longer than the specified 0
Created a chunk of size 1087, which is longer than the specified 0
Created a chunk of size 1188, which is longer than the specified 0
Created a chunk of size 303, which is longer than the specified 0
Create

In [8]:
doc[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gil

In [12]:
db_books = Chroma.from_documents(
    doc,
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
query = " a book to teach children about nature"
document = db_books.similarity_search(query,k=10)
document

[Document(id='a594d274-14d7-4d60-80dc-a164eb4f89cb', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='21e78889-d734-44d3-9b19-1239b0323ab6', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='3e164002-f9ed-486e-8b19-213232170f32', metadata={'source': 'ta

In [18]:
def retrieve_recommendation(query:str,top_k:int) -> pd.DataFrame:
    documents = db_books.similarity_search(query,k=top_k)
    book_list =[]
    for i in range(0,len(documents)):
        book_list +=[int(documents[i].page_content[:13])]
    return books[books['isbn13'].isin(book_list)].head(top_k)

In [24]:
list_books = retrieve_recommendation(query,top_k=10)
list_books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_&_desc,tagged_description
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal SummerA Novel,9780060959036Barbara Kingsolver's fifth novel ...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453‘Racso, a brash and boastful litt..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870When Margaret and her younger bro...
1642,9780374522599,0374522596,The Control of Nature,John McPhee,Nature,http://books.google.com/books/content?id=p1qKQ...,The Control of Nature is John McPhee's bestsel...,1990.0,4.24,288.0,3365.0,The Control of Nature,9780374522599The Control of Nature is John McP...
2216,9780440421702,0440421705,Hoot,Carl Hiaasen,Bullying,http://books.google.com/books/content?id=uKUTo...,"Roy, who is new to his small Florida community...",2006.0,3.82,292.0,83557.0,Hoot,"9780440421702Roy, who is new to his small Flor..."
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069Children will discover the exciti...
3748,9780786808373,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,Baby Einstein: Birds,"9780786808373Introducing your baby to birds, c..."
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380Introduce your babies to birds, c..."
3750,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,"9780786808397Introduce your baby to birds, cat..."
3765,9780786819119,0786819111,"Baby Einstein: Water, Water Everywhere","Disney Book Group,",Juvenile Fiction,http://books.google.com/books/content?id=tuAdA...,Charming illustrations and playful rhythmic ve...,2003.0,3.70,10.0,77.0,"Baby Einstein: Water, Water Everywhere",9780786819119Charming illustrations and playfu...


In [28]:
list_books[['title','description']]

,title,description
324,Prodigal Summer,Barbara Kingsolver's fifth novel is a hymn to ...
404,Racso and the Rats of NIMH,"‘Racso, a brash and boastful little rodent, is..."
406,"R-T, Margaret, and the Rats of NIMH","When Margaret and her younger brother, Artie, ..."
1642,The Control of Nature,The Control of Nature is John McPhee's bestsel...
2216,Hoot,"Roy, who is new to his small Florida community..."
3747,Baby Einstein: Neighborhood Animals,Children will discover the exciting world of t...
3748,Baby Einstein: Birds,"Introducing your baby to birds, cats, dogs, an..."
3749,Baby Einstein: Babies,"Introduce your babies to birds, cats, dogs, an..."
3750,Baby Einstein: Dogs,"Introduce your baby to birds, cats, dogs, and ..."
3765,"Baby Einstein: Water, Water Everywhere",Charming illustrations and playful rhythmic ve...
